# Notes

Picking up from Jan 13, 2015, we have some questions to ask about the data reduction thus far to assess its accuracy. Here are some questions I hope to answer: 

* Does the nod-correlated red noise in the high-noise spectral bins have anything to do with the galaxy subtraction (which is nod dependent)? If so, fix it.

* Does marginalizing over my uncertainties (including a fitting parameter that varies the uncertainties) change the best-fit spectrum/posteriors significantly (better account for red-noise without modelling it directly)?

* Does including a red-noise model (gaussian processes) improve the fits noticeably? Are the results from such a model believable?

* Does tweaking the aperture size or the background subtraction annulus around my aperture improve photometric precision at all?


##### Notes on files: 

Each `phot_fit*.ipynb` file sets up an MCMC run, each `fit-results_*.ipynb` analyzes the results of the corresponding `phot-fit` file.


### phot_fit-finetime.ipynb

The most recently completed notebook on fitting the light curves appropriately, with a detrending term for airmass and properly binning in time.

### /astro/users/bmmorris/git/research/keck/2014september/analysis/rightnod/galaxycut.ipynb

To address the first bullet -- is the "galaxy stamp" a good image to use? Are there bad pixels left in the image that can still be removed (even if manually)?

I found some bad pixels left in the image. None of them were tremendous or particularly bad, but I removed them nonetheless and found a bug in the way that bad pixels are corrected: always do the `if x_badpxl-window < 0:` check!!!

Also discovered that there was no way to make the "wholeframe" galaxy image --> where the galaxy stamp is put into a zeroed image. Adding that to galaxycut.ipynb now.

### phot_fit-bpcorr.ipynb

Re-doing the fits in phot_fit-finetime.ipynb, but this time fixing the above bug in bad pixel fitting and using the updated galaxy stamp (with less bad pixels). Then start fitting with init parameters being the last best parameters from `phot_fit-finetime.ipynb`: "max_lnp_params_201412310852.npy".

### phot_fit-modgalaxy.ipynb

The flux of the galaxy should be changing from image to image due to telluric effects, just as the flux from the comparison star varies. As a result, let's hold the ratio of comparison star flux to galaxy flux constant by: 

* Make a summed composite of the comparison star and galaxy which we can use to find the ratio of fluxes between comparison star and galaxy in each spectral bin (implemented this all in galaxycut.ipynb)

* In the photometry loop in phot_fit-modgalaxy.ipynb, after you measure the flux of the comparison star, scale the galaxy correction image's flux by the ratio of comparison-to-galaxy flux found in the previous step for each spectral bin.

* Add in the scaled image of the galaxy before measuring the flux of the target star

### phot_fit-galaxybp.ipynb

After updating the galaxy correction in the above file, I noticed that there were still plenty of bad pixels in the core of the PSF. These bad pixels went uncorrected because they appear to be pixels with a different sensitivity response than neighboring pixels. To correct these pixels, I created a series of bad pixel search notebooks in `/astro/users/bmmorris/git/research/keck/2014september/analysis/rightnod/bpsearch-*.ipynb` which experiment with taking the sum of all nod-subtracted frames and looking for pixels that don't match their neighbors. 

The routine I've settled with is **`rightnod/bpsearch-fft.ipynb`**, which uses a FFT to reconstruct the summed image with only low frequency terms, then uses a difference of the FFT image from the original image to find sharp pixels discontinuities. This routine produces a bad pixel mask, and in it I define an image correction method which takes the bad pixel mask and corrects each bad pixel with the median of a few neighboring pixels (in the spectral direction).

In `phot_fit-galaxybp.ipynb` I will implement this new correction method in the photometry routine.

The new light curves don't look much different, so I'm going to make a new copy of `phot_fit-bpcorr.ipynb`...

### phot_fit-bpcorr2.ipynb

Update to `phot_fit-bpcorr.ipynb` with new bad pixel correction in the core of the PSF from `rightnod/bpsearch-fft.ipynb`. The photometry looks pretty good! Waiting on the fits, being reduced with `fit_results-bpcorr2.ipynb`

### phot_fit-bpcorr3.ipynb

What happens if you remove the chunk of the photometry script that removes the highly variable pixels (the original bp correction, implemented in `phot_fit-bpcorr.ipynb`) and instead just leave in the section that corrects the cores of the PSFs (implemented in `phot_fit-bpcorr2.ipynb`)? Turns out, not much. Its hard to notice the differences in the light curves even when flipping back and forth between them. I'll stick with `phot_fit-bpcorr2.ipynb` for now.

***

### phot_fit-spitz.ipynb

After receiving the Spitzer PLD light curves from Sarah Ballard, it's now time to add them into the reduction. `phot_fit-spitz.ipynb` will begin as a copy of `phot_fit-bpcorr2.ipynb`.

###  phot_fit-spitzwhitekernel.ipynb

After using TAP to find red-noise in the Spitzer observations, I find that the errorbars on Rp/Rs found by `phot_fit-spitz.ipynb` are underestimated when compared with the TAP results. For that, I'm introducing a white kernel and letting the errorbars float on the data. 

It looks like letting the Spitzer errors float with a white kernel isn't making the uncertainties on Rp/Rs in the Spitzer bands any larger. New idea: let all uncertainties float:

###  phot_fit-spitzwhitekernelall.ipynb

After using TAP to find red-noise in the Spitzer observations, I find that the errorbars on Rp/Rs found by `phot_fit-spitz.ipynb` are underestimated when compared with the TAP results. Now I'm using white kernels for ALL light curves, including for MOSFIRE light curves.

### phot_fit-latestorb.ipynb

In this iteration I'm going to update the fixed orbital parameters in the fit. For example, I realized the argument of periastron $\omega \neq \pi/2$ and the period that I was using were not from the most recent available references. I'm also going to be sure to overdisperse the white noise kernel hyperparameter, since I know that all of the hyperparameters were started at their final values in the `phot_fit-spitzwhitekernelall.ipynb` run. This seems like I'm using an incompatible set of orbital parameters that cause the planet not to transit. 

### phot_fit-redwhite.ipynb

In this iteration I'm adding a matern kernel to the george noise model. -- This has too many parameters and doesn't seem to be running through emcee right.

### phot_fit-mosredonly.ipynb

Here I'll try to fit the MOSFIRE residuals with a red noise model. First results: when modeling with white+Matern32 kernel, the white noise kernel still gets all of the noise, the Matern kernel gets none of the amplitude.


### phot_fit-mosexpsin.ipynb

Here I'll try to fit the MOSFIRE residuals with a red noise model: using white+ExpSine2Kernel. Doesn't converge.

### characterizingnoise.ipynb

I haven't done half of the standard red-noise characterization schemes. Time to get on that. The autcorr shows what appears to be a gaussian*cosine kernel should do the trick. Try that.

### phot_fit-mossqexpcos.ipynb

in `characterizingnoise.ipynb` I found that a (squared exponenetial)*(cosine) kernel best describes the MOSFIRE data (in some channels). Making a copy of `phot_fit-mosredonly.ipynb` and implementing that kernel.

### phot_fit-spitzsqexpcos.ipynb

in `characterizingnoise.ipynb` I found that a (squared exponenetial)*(cosine) kernel best describes the Spitzer Ch1 data (in some channels). Making a copy of `phot_fit-mossqexpcos.ipynb` and implementing that kernel, this time for the Spitzer data.

### phot_fit-MpinkSwhite.ipynb

Though I can't yet get the GP to pick up on the correlations in the Spitzer Ch1 LC in `phot_fit-spitzsqexpcos.ipynb`, I'm moving forward and creating an update to `phot_fit-spitzwhitekernelall.ipynb` which implements the (sq exp)*(cosine) kernel on the MOSFIRE data, in the hopes that whenever I get my act together with the Spitzer data I'll already have converged chains on the MOSFIRE data with pink noise.

Also reverting back to $e=\omega=0$ (Husnoo 2012).

### phot_fit-MpinkSwhiteHODLR.ipynb

While MpinkSwhite.ipynb is running, this run will use the HOLDR solver to see if it's stable. Compare its results to the non-HOLDR solutions.

### phot_fit-MpinkSwhite.py

I've gotten working with the many-cored machine `magneto`, and to submit jobs to magneto I've converted the notebook `phot_fit-MpinkSwhite.ipynb` into a python script. It doesn't work on `magneto` with the command `python` for path issue reasons, so simply run it with `ipython` instead.

### reparameterizelc.ipynb

The fits in `phot_fit-MpinkSwhite.ipynb` are looking good, but the correlations between $a/R_*$ and $i$ are causing the chains to sample the parameter space inefficiently. To correct this, I should sample $T_{14}$ and $b$ instead. In this notebook, I'll re-write the transit model function to accomadate that, and maybe do some of the operations on old chains to determine where to start new chains with the new parameterization.

### phot_fit-tbMpinkSwhite.py

This new parameterization ($T,b$ in place of $a/R_s, i$) in `reparameterizelc.ipynb` looks like it works great on the samples from `phot_fit-MpinkSwhite.ipynb`. In this version of the python script I'm going to update the transit model to use the new reparameterization. 

### phot_fit-tbMpinkSwhite2.py

I've sent an update to Sarah with the results from phot_fit-tbMpinkSwhite.py so we can recompute PLD on the Spitzer light curves with the updated parameters from my joint fit. In the mean time, the phot_fit-tbMpinkSwhite.py chains are nearly converged, but a lot of walkers are stuck. I'm going to restart the chains by sampling from the current position of the chains.


### phot_fit-tbMpinkSwhitePLD3.py

Sarah has twice iterated on the PLD light curves now, and this third version is probably the final one. I'm going to implement a version of the `binned_lc()` method that checks to see if the difference between exposure times is less than 5 seconds, and if so, it will not make a binned light curve and skip right to `reparameterized_lc()`. This makes sense for these unbinned Spitzer light curves that have 2-second exposures. I've implemented a new data storing scheme that saves chains in a new directory made for each run, located at `/astro/store/scratch/tmp/bmmorris/longchains/mosfirespitzer/`. It also now saves the acceptance rate for each chain and the autocorrelation length (when available).

After reading up on DFM's emcee paper, I'm going to up the number of walkers as well. This is a potential remedy for getting stuck chains unstuck. In addition, I'll increase the spread of the initial parameters.

Initial run is stored in `phot_fit-tbMpinkSwhitePLD3201503102255/` on scratch. This run begins the walkers by looking at an old nearly-converged run, and redispersing the walkers by a factor of three times their old standard deviation. These overdispersed walkers are not helping. My hypothesis is: since there are 64 parameters, sampling an overdispersed population of walkers is bound to yield low acceptance rates because there are so many walkers and some of them are bound to start off in bad parts of parameter space. 

### phot_fit-tbMpinkSwhitePLD3_nogeorge.py

Since that wasn't really working either, I decided to scrap red noise analysis for a minute and get a good fit again in `phot_fit-tbMpinkSwhitePLD3_nogeorge201503111238`. I've used basic $\chi^2$ for the lnlike(), and fit the MOSFIRE+Spitzer light curves with 500 walkers. This should converge by ~tomorrow morning. Then, restart the run with red noise analysis.

Looks like the chains converged pretty well in `phot_fit-tbMpinkSwhitePLD3_nogeorge201503111238`. The  airmass constant $c_X$ posteriors for the three longest wavelength bins look relatively bimodal -- watch that and make sure it's not ruining convergence. 

I'm going to now update **`phot_fit-tbMpinkSwhitePLD3.py`** to use the outputs of `phot_fit-tbMpinkSwhitePLD3_nogeorge.py` in order to start the chains in the appropriate part of parameter space.

### phot_fit-tbMpinkSwhitePLD3_nored.py

Trying one layer deeper than `tbMpinkSwhitePLD3_nogeorge`, and adding white noise modeling back in, but not red.


### phot_fit-tbMpinkSwhitePLD3_mult.py

Rodrigo says sometimes george is unstable for small numbers. Create a function that multiplies the residuals by 1e6, then pass to george. Starting with version `phot_fit-tbMpinkSwhitePLD3_nored`


#### ...and then there was a storm of trials on phot_fit-tbMpinkSwhitePLD3_mult.py
There was a flurry of experimentation that occurred while working to make `phot_fit-tbMpinkSwhitePLD3_mult.py` converge to the right values. First, I turned off all calls to george and computed lnlike using $\chi^2$. Once I showed that I could converge to the right values, I then turned on white noise only with george. I experimented with scaling my residuals by a constant factor -- at first 1e6, then 1e4 -- and I found that by scaling my residuals by 1e4 I could get george to converge on white noise solutions. Then I experimented with adding red noise. In that iteration, I found that I was starting my walkers far from the solutions, and it would take a very long time for the under-dispersed and incorrect red noise hyperparams to converge, so I killed the run and started the run again with over-dispersed red-noise hyperparams. This run seems like it's working!


```python
rundir = '/astro/store/scratch/tmp/bmmorris/longchains/' + \
          'mosfirespitzer/phot_fit-tbMpinkSwhitePLD3_mult201503141253/' # Check with chi2 that worked
    
rundir = '/astro/store/scratch/tmp/bmmorris/longchains/' + \
          'mosfirespitzer/phot_fit-tbMpinkSwhitePLD3_mult201503141328/' # Check w white noise only that worked
rundir = '/astro/store/scratch/tmp/bmmorris/longchains/' + \
         'mosfirespitzer/phot_fit-tbMpinkSwhitePLD3_mult201503141429/' # Works w/ red noise, far from convergence though
    
rundir = '/astro/store/scratch/tmp/bmmorris/longchains/' + \
         'mosfirespitzer/phot_fit-tbMpinkSwhitePLD3_mult201503141456/' # Restart with broader red noise amp
```

Though it was indeed working, it's taking FOREVER to converge. In this next trial `phot_fit-tbMpinkSwhitePLD3_selective.py`, based on `phot_fit-tbMpinkSwhitePLD3_mult.py`, select which MOSFIRE bands to apply the red noise search to.

WAIT -- it looks like `phot_fit-tbMpinkSwhitePLD3_mult.py` might not have been fitting for red noise properly after all. Correcting the mistake in lnlike() in `phot_fit-tbMpinkSwhitePLD3_mult_corrected201503170936/`.

Selective isn't working, stopping corrected to restart corrected with many threads.

### phot_fit-clean.py

I found that no matter how long the above runs were going, they would not converge. The $\ln p$ would continue to increase, and I couldn't figure out what to do to improve the initial positions of the walkers to yield a better run. 

To start fresh without reinventing the wheel, I've migrated the contents of `phot_fit-tbMpinkSwhitePLD3_mult_corrected.py` to `phot_fit-clean.py` and the `moar` package. The motivations for doing this include: 

* Readability and organization. The old code was becoming to cumbersome and large to manage. 

* Versatility with repeatability. Before I would go in and change all of the parameters by hand if I was switching noise models, changing the log likelihood each time. In order to ensure that runs are repeatable, `moar` will contain a collection of all possible lnlike() functions, with toggle options that switch between them



***
## Things to double check:

* Compare all best fit results with Jordan/Nikolov. Changing $a/R_s$ or $i$ can bias the Rp/Rs values. 

* What happens if we fix $a/R_s$ to values from Jordan/Nikolov/Dragomir? That can remove bias in Rp/Rs comparison

* Do the mid-trans times agree with linear ephemeris (1) when compared with earlier results, (2) are they self-consistent between MOSFIRE + Spitzer?

* What are the Claret LD parameters for this target, are they consistent with fits? Claret's Spitzer LD parameters don't seem to match the data (or even match limb-*darkening* constraint). Double check that the claretld module is accurately recovering these parameters

* Check if TAP recovers similar parameters on the MOSFIRE data

* Fix eccentricity? Prior? Ah!? **WAIT THE ARGUMENT OF PERIASTRON $\neq \pi/2$ FOR $e\neq 0$**

* Set up quick run with red noise using the residuals from a fit to the MOSFIRE observations -- is the red noise component very strong using Matern kernel?

* ~~Do spectral cross correlation throughout night to measure in-slit shifts. Compare to the shifts in spatial direction~~ (see bothnods/checks/shiftsinslit.ipynb)

* How does the BP mask generated by running the FFT algorithm on the flat field compare to the BP mask from the data images?

***
## Thoughts for future directions

* when you look at the summed, nod-subtracted images you can see some pixels in the core of the PSF that just aren't measuring fluxes, and these pixels aren't being corrected by my algorithm right now. This might be significant and can contribute the nod-correlated flux changes taht we're measuring. I think I should go back into my bad pixel detection routines and add a new package into it that measures the brightness of pixels in the core of the PSF of the big added composite image and marks off outliers that should be replaced in each image with the interpolation of the flux on either side (in the spectral direction -- while this is dangerous to do on a large scale, we can be safe in doing so if we ensure that it only ever corrects single pixels here and there).

* Tests of linearity: isn't measuring the relative brightnesses of the target/comp star a fine test of linearity? They're the brightest pixels in the exposures, and the difference in magnitudes should be well measured in the K band for all of our stars... let's just see if $\Delta$mag is what we expect.